# Collect Tweets in MongoDB

In [1]:
!pip install pymongo

     |████████████████████████████████| 506 kB 42.6 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install pymongo[srv]

     |████████████████████████████████| 241 kB 43.3 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install dnspython

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install tweepy

     |████████████████████████████████| 64 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 146 kB 20.6 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [5]:
!pip install twitter

     |████████████████████████████████| 50 kB 5.7 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


# Import Python Libraries

In [6]:
import pymongo
from pymongo import MongoClient
import json
import tweepy
import twitter
from pprint import pprint
import configparser
import pandas as pd

# Load Authorization Info

In [11]:
config = configparser.ConfigParser()
config.read('config.ini')

CONSUMER_KEY      = config['mytwitter']['api_key']
CONSUMER_SECRET   = config['mytwitter']['api_secrete']
OAUTH_TOKEN       = config['mytwitter']['access_token']
OATH_TOKEN_SECRET = config['mytwitter']['access_secrete']

mongod_connect = config['mymongo']['connection']

In [12]:
client = MongoClient(mongod_connect)
db = client.gp25 # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'id_1'

# Use restAPI to collect tweets

In [13]:
rest_auth = twitter.oauth.OAuth(OAUTH_TOKEN,OATH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
rest_api = twitter.Twitter(auth=rest_auth)

In [18]:

count = 100 #number of returned tweets, default and max is 100
#geocode = "38.4392897,-78.9412224,50mi"  # defin the location, in Harrisonburg, VA
q = "covid19"                               #define the keywords, tweets contain election

In [24]:
search_results = rest_api.search.tweets( count=count,q=q) #you can use both q and geocode
statuses = search_results["statuses"]
since_id_new = statuses[-1]['id']
for statuse in statuses:
    try:
        tweet_collection.insert_one(statuse)
        pprint(statuse['created_at'])# print the date of the collected tweets
    except:
        pass

'Thu Nov 04 19:19:50 +0000 2021'
'Thu Nov 04 19:19:50 +0000 2021'
'Thu Nov 04 19:19:49 +0000 2021'
'Thu Nov 04 19:19:49 +0000 2021'
'Thu Nov 04 19:19:49 +0000 2021'
'Thu Nov 04 19:19:47 +0000 2021'
'Thu Nov 04 19:19:46 +0000 2021'
'Thu Nov 04 19:19:46 +0000 2021'
'Thu Nov 04 19:19:46 +0000 2021'
'Thu Nov 04 19:19:46 +0000 2021'
'Thu Nov 04 19:19:46 +0000 2021'
'Thu Nov 04 19:19:45 +0000 2021'
'Thu Nov 04 19:19:45 +0000 2021'
'Thu Nov 04 19:19:43 +0000 2021'
'Thu Nov 04 19:19:42 +0000 2021'
'Thu Nov 04 19:19:42 +0000 2021'
'Thu Nov 04 19:19:42 +0000 2021'
'Thu Nov 04 19:19:40 +0000 2021'
'Thu Nov 04 19:19:40 +0000 2021'
'Thu Nov 04 19:19:40 +0000 2021'
'Thu Nov 04 19:19:39 +0000 2021'
'Thu Nov 04 19:19:39 +0000 2021'
'Thu Nov 04 19:19:38 +0000 2021'
'Thu Nov 04 19:19:37 +0000 2021'
'Thu Nov 04 19:19:36 +0000 2021'
'Thu Nov 04 19:19:34 +0000 2021'
'Thu Nov 04 19:19:33 +0000 2021'
'Thu Nov 04 19:19:33 +0000 2021'
'Thu Nov 04 19:19:32 +0000 2021'
'Thu Nov 04 19:19:32 +0000 2021'
'Thu Nov 0

In [26]:
since_id_old = 0
while(since_id_new != since_id_old):
    since_id_old = since_id_new
    search_results = rest_api.search.tweets( count=count,q=q,
                        geocode=geocode, max_id= since_id_new)
    statuses = search_results["statuses"]
    since_id_new = statuses[-1]['id']
    for statuse in statuses:
        try:
            tweet_collection.insert_one(statuse)
            pprint(statuse['created_at']) # print the date of the collected tweets
        except:
            pass

# View collected tweets

In [27]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

424
323
